# 01 - Python and Pandas: Summarizing Voter List Data
<p class="lead">
Michelle Brown Notes v 1.7<br>

In the previous notebook we examined a familar and small dataset: a csv file with a list of a about 21 thousand polling stations. And now we know how to view and filter the data and how to do pivot tables.  
In this notebook we are going to be combining files into a much larger file--a voter list of more than 5 million records. Such a large file is incredibly hard and slow if we try to analyze it in excel. You'll see the power of Python in this notebook as you manipulate the 5 million record file withouth having to wait hours between each task. 
The three files we'll be using are a scrape of the voters list on the Azerbaijan's Central Election Commission website. We'll learn how how to combine the three files into one, and we'll begin to describe and summarize the data, and make a simple histogram. We'll also go over how to save subsets of the data (like duplicate names).

# Outline

<!-- MarkdownTOC autolink=true autoanchor=true bracket=round -->

- [Import Libraries](#imp)
- [Read in the first file](#read1)
- [Combining three csv files to 1 master file](#comb)
- [Checking missing values](#missing)
- [Snapshot of histograms (plots)](#plots)
- [Duplicate and unique names](#dups)
- [Summarizing at a higher level](#sumhigh)
- [Create a dataset at the polling station level](#pslevel)
- [Make a histogram](#histo)
- [Save the polling station level dataset](#save)

<!-- /MarkdownTOC -->

<a name="imp"></a>
# Importing libraries

Again we import the analysis module called pandas as a variable called 'pd' so we can use it's associated methods. But we are also going to import some other libraries that we'll use later to make inline plots. 

In [ ]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

## Download the three Voter List files

Download the three files from this <a href="https://www.dropbox.com/sh/zu2lb89i0nefqfl/AACMpOC2Ezk5GHbCJWTfOQOsa?dl=0">folder</a> and save them into your data folder on your computer.  

<a name="read1"></a>
# Read in first file of the Voter List

We read in the csv file and store it as dataframe called df1 and see it's shape (rows and columns)

In [ ]:
df1 = pd.read_csv('data/AZVotersList2016_part_1.csv')
df1.shape

Look at the column names

In [ ]:
df1.columns

In [ ]:
df1.head()

<a name="comb"></a>
# Combining three datasets into one master 

Read in the second file as a datafrace called df2 and see how many rows (1,478,639) and columns (8) it has:

In [ ]:
df2 = pd.read_csv('data/AZVotersList2016_part_2.csv')
df2.shape

Let's look at the columns to check it's the same structure (order) as the first dataset

In [ ]:
df2.columns

In [ ]:
df2.head()

When we concatenate DataFrames, we need to specify the axis. axis=0 tells Pandas to stack the second DataFrame under the first one. It will automatically detect whether the column names are the same and will stack accordingly. axis=1 will stack the columns in the second DataFrame to the RIGHT of the first DataFrame. To stack the data vertically, we need to make sure we have the same columns and associated column format in both datasets. When we stack horizonally, we want to make sure what we are doing makes sense (ie the data are related in some way).

In [ ]:
# stack the DataFrames on top of each other
df_12 = pd.concat([df2, df1], axis=0)
df_12.shape

In [ ]:
# If we had wanted to place the DataFrames side by side, we would have used this instead (remove the # to uncomment the code)
#horizontal_stack = pd.concat([df2, df1], axis=1)

In [ ]:
df_12.head

Because of the stacking, the index is not correct (notice how the final row looks like it is 2110684) so we need to reset the index. 

In [ ]:
df_12 = df_12.reset_index()
del df_12['index']

Notice that the last row now has the proper index (3589323)

In [ ]:
df_12.head

<b> Adding the third dataset</b><br>
Now let's add the third datset to the combination of the first and second (repeating steps above)

We read in the third file and see it has 1550088 records and again the 8 columns)

In [ ]:
df3 = pd.read_csv('data/AZVotersList2016_part_3.csv')
df3.shape

Check the columns are the same and in same order as the other files: 

In [ ]:
df3.columns

Add the third dataset to the bottom of the combined (file 1 & 2) dataset

In [ ]:
# stack the DataFrames on top of each other
df_all = pd.concat([df3, df_12], axis=0)
df_all.shape


In [ ]:
df_all = df_all.reset_index()
del df_all['index']

In [ ]:
df_all.tail

<a name="missing"></a>
# Checking the missing values on the complete dataset

This will give use the number of missing (null) values for each column

In [ ]:
df_all.isnull().sum()

This will give us a count of non-null values for each column 

In [ ]:
df_all.count()

<h2> Dealing with birthdate</h2>

We have the Year_of_Birth variable. Let's check it's data type and then convert it to age. 

Let's take a look at the Year_of_Birth variable and not the data type:

In [ ]:
df_all['Year_of_Birth']

The data type is "int64" which means it is an integer (i.e., a number not a string of text) so this is actually an easy case because we are not converting a complex date type with day month and year. 

Let's create a new variable called age by subtracting Year_of_Birth from 2017:

In [ ]:
df_all['Age'] = 2017 - df_all['Year_of_Birth']

In [ ]:
df_all['Age']

In [ ]:
df_all["Age"].describe()
#df_all["Age"].min()

In [ ]:
df_all.columns

<a name="plots"></a>
# Snapshot of histograms (plots)

Let's make a simple histogram of Age

In [ ]:
df_all.hist('Age')

Let's say we want more bins so we get a more granular sense of the distribution. We can specify the number of bins:

In [ ]:
df_all.hist(column = 'Age', bins = 20)

The default options for plots are pretty ugly. This code will applying one of the styles available called ggplot to all of our next plots.  It incorporates many of the best practices for presenting data and focus on simplicity. It is modeled after the plotting system in the R language. 

In [ ]:
plt.style.use('ggplot')

In [ ]:
df_all.hist(column = 'Age', bins = 20)

If you want to see all the available styles, run the code below. Here is a link to gallery so you can see how they look: https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html

In [ ]:
plt.style.available

The code below shifts the look of all the plots to mirror the 538 blog (previously at the New York Times):

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
df_all.hist(column = 'Age', bins = 20)

Or if you want to apply the style to just the one plot (and not all of the plots):

In [ ]:
with plt.style.context('seaborn-colorblind'):  #add this line to specify just this plot
    df_all.hist(column = 'Age', bins = 20)

You can also change the title to a specific text and style it (last line below) using the plt.title method and parameters: 

In [ ]:
with plt.style.context('bmh'):  
    df_all.hist(column = 'Age', bins = 20)
    plt.title("Other Title", fontname='Arial', fontsize=16, fontstyle='italic', fontweight='bold', color='darkblue')

<a name="dups"></a>
#  Duplicate and unique names</h1>
When we're analyzing voter list data we often want to check for duplicate names or duplicate voter id numbers when we have those variables (we only have names in the Azerbaijan dataset).  Let's check for the number of unique entries in specific column 'Name' (and temporarily put this in a series).

In [ ]:
namecount = df_all.groupby('Name').size()
namecount.shape

In [ ]:
#create a new dataframe from the series with each name and also create a new column with number of times that name is listed
nameunique = pd.DataFrame(namecount, columns = ['size'])

In [ ]:
nameunique.shape

In [ ]:
nameunique.head

So it looks like we have 4,650,072 unique entries for Name 

Now we can create a dataset that lists only duplicate names (i.e., where size is greater than 1) and the number of times they are duplicated 

In [ ]:
namedup = nameunique[nameunique['size'] > 1]

In [ ]:
namedup.shape

There are 303,449 entries in the name column that appear 2 or more times. 

In [ ]:
namedup.head

Let's save this smaller list to a csv file (tab separated) so you can import it into excel and look at it. 

In [ ]:
namedup.to_csv('Name_duplicates_wcounts_encode.csv', sep='\t', encoding='utf-8')

<a name="sumhigh"></a>
# Summarizing at a higher level

### Let's count how many names (non blank) are in each Constitutency

In [ ]:
counts = df_all.groupby('Constituency').size(); counts

<a name="pslevel"></a>
# Create a polling station level dataset

Create a new variable that makes a unique code for the polling station by adding the Constituency to the Precinct Number

In [ ]:
df_all['PSunique'] = df_all.Constituency.astype(str).str.cat(df_all.Precinct_Number.astype(str), sep='_')

Now let's get the counts for each polling station using the unique code we just created:

In [ ]:
pscounts = df_all.groupby('PSunique').size(); pscounts

Look at the shape of our data

In [ ]:
pscounts.shape

Uh oh. What we created is actually a series (all in one column) but we want to make it a dataframe so we can label the columns. So we convert it the series to a dataframe (an we reset the dataframe index).

In [ ]:
pscounts = pscounts.to_frame().reset_index()

Let's confirm it's 2 columns:

In [ ]:
pscounts.shape

Yay, Let's look at the names of our columns

In [ ]:
pscounts.columns

Re-name the second column (currently labeled 0) to 'numVoters'

In [ ]:
pscounts.rename(columns={0: 'numVoters'}, inplace=True)

In [ ]:
#check it
pscounts.columns

Let's get a quick view of the summary statistics for the numerical data (i.e., numVoters) in this polling station level dataset:

In [ ]:
pscounts.describe()

The output you get might be in scientific notation, if so let's run this bit of code to alter the display and then try describe again:

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
pscounts.describe()

<a name="histo"></a>
# Make a histogram of Voters (Polling station dataset)

In [ ]:
pscounts.hist('numVoters');

In [ ]:
pscounts["numVoters"].plot(kind="hist")

<a name="save"></a>
# Save the polling station level dataset

Save this polling station-level dataset to a csv file that is tab separated and with utf-8 encoding

In [ ]:
pscounts.to_csv('PollingStation_counts.csv', sep='\t', encoding='utf-8')

Or we could save it to a csv file that is (by default) comma separated by not including the sep='\t' option

In [ ]:
pscounts.to_csv('PollingStation_counts_comma.csv', encoding='utf-8')

This is the end of this notebook. What other things would you want to analyze in a voter list?